# Computational Tests

This notebook is intended for running different kinds of analyses that would validate our reconstruction and would compare it woth previous CHO Recons

## 1.Blocked reactions in each subsystem subsystem
Here we calculate the percentage of blocked reactions in each one of our subsystems and then we compare it with previous reconstructions (iCHO1766 and iCHO2101). To calculate this value we use the .txt file generated in the **Final CHO Model** notebook **Section 2.1**

In [ ]:
import pandas as pd
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt

from cobra.io import load_json_model

In [ ]:
model = load_json_model('iCHOv3_CHO_09022024.json')

### 1.1 Calculate the percentage of blocked reactions per Subsystem

In [ ]:
blocked_reactions = pd.read_csv('temp/Blocked_Reactions.csv')
blocked_reactions = list(blocked_reactions['Reaction'])

In [ ]:
##### ----- Print the amount and total amount of blocked reactions ----- #####            
print('##### ----- Blocked Reactions ----- #####')
print(f'The model has {len(model.reactions)} total reactions')
print(f'The model has {len(blocked_reactions)} ({round(len(blocked_reactions)/len(model.reactions)*100)}%) blocked reactions')

In [ ]:
#### --- Calculate the percentage of blocked reactions per Subsystem ---- ####
blocked_subs = []
for blocked in blocked_reactions:
    blocked_rxn = model.reactions.get_by_id(blocked)
    blocked_subs.append(blocked_rxn.subsystem)
    
counter_blocked = Counter(blocked_subs)

all_subs = []
for rxn in model.reactions:
    all_subs.append(rxn.subsystem)
counter_all = Counter(all_subs)
counter_all

# calculating percentages
percentages = {}

for subsystem in counter_all.keys():
    if subsystem in counter_blocked:
        percentages[subsystem] = (counter_blocked[subsystem] / counter_all[subsystem]) * 100
    else:
        percentages[subsystem] = 0
        
# Sort the dictionary by value in descending order
sorted_percentages = {k: v for k, v in sorted(percentages.items(), key=lambda item: item[1], reverse=True)}

for k, v in sorted_percentages.items():
    print(f"{k}: {v}%")

### 1.2 Compare and plot the percentage of blocked reactions between iCHO1766, iCHO2101 and our reconstruction

In [ ]:
icho1766_icho2101 = pd.read_excel('../Data/Reconciliation/datasets/iCHO2101.xlsx', 'Supplementary Table 9', header = 1)
icho1766_icho2101['Name of the metabolic pathway'] = icho1766_icho2101['Name of the metabolic pathway'].str.replace("'","")
# Now converting the dataframe to dictionary
data_dict = icho1766_icho2101[['Percent of blockage in iCHO1766', 'Percent of blockage in iCHO2101']].to_dict()
pathway_names = list(icho1766_icho2101['Name of the metabolic pathway'])

# your original dictionaries
blockage_iCHO1766 = data_dict['Percent of blockage in iCHO1766']
blockage_iCHO2101 = data_dict['Percent of blockage in iCHO2101']

# initialize new dictionaries
new_blockage_iCHO1766 = {}
new_blockage_iCHO2101 = {}

# fill the new dictionaries with the correct keys
for i, pathway in enumerate(pathway_names):
    new_blockage_iCHO1766[pathway] = blockage_iCHO1766[i]
    new_blockage_iCHO2101[pathway] = blockage_iCHO2101[i]

In [ ]:
# initialize lists to hold subsystems with lower and higher values in percentages_filtered
lower_subsystems = []
higher_subsystems = []

# loop through each subsystem
for subsystem in pathway_names:
    # get the value from each dictionary, or default to 0 if the subsystem is not present
    value_percentages = percentages.get(subsystem, 0)
    value_iCHO1766 = new_blockage_iCHO1766.get(subsystem, 0)
    value_iCHO2101 = new_blockage_iCHO2101.get(subsystem, 0)
    
    # check if value_percentages_filtered is lower than both of the other values
    if value_percentages < min(value_iCHO1766, value_iCHO2101):
        lower_subsystems.append(subsystem)
    
    # check if value_percentages_filtered is higher than both of the other values
    if value_percentages > max(value_iCHO1766, value_iCHO2101):
        higher_subsystems.append(subsystem)

# print the results
print("Subsystems where 'percentages_filtered' is lower:")
for subsystem in lower_subsystems:
    print(subsystem)

print("\nSubsystems where 'percentages_filtered' is higher:")
for subsystem in higher_subsystems:
    print(subsystem)


In [ ]:
# List of subsystems for which we want to plot histogram
subsystems = lower_subsystems

# Create new lists with the percentages for each subsystem in each dictionary
values_icho1766 = [new_blockage_iCHO1766.get(subsystem, 0) for subsystem in subsystems]
values_icho2101 = [new_blockage_iCHO2101.get(subsystem, 0) for subsystem in subsystems]
percentages_values = [percentages.get(subsystem, 0) for subsystem in subsystems]

# Define the width of each bar and the positions of the bars
bar_width = 0.2
index = np.arange(len(subsystems))

# Create the bar plot
fig, ax = plt.subplots()

bar1 = ax.bar(index, values_icho1766, bar_width, label='iCHO1766')
bar2 = ax.bar(index + bar_width, values_icho2101, bar_width, label='iCHO2101')
bar3 = ax.bar(index + 2*bar_width, percentages_values, bar_width, label='iCHO3000')

ax.set_xlabel('Subsystems')
ax.set_ylabel('Percentage of Blocked Reactions (%)')
ax.set_title('Percentage of Blocked Reactions for Each Subsystem')
ax.set_xticks(index + bar_width)
ax.set_xticklabels(subsystems, rotation=90)
ax.legend()

plt.show()


#### Auxotrophies

In [ ]:
from cobra.io import load_json_model
iCHO_path = "iCHOv3_CHO_09022024.json"
iCHO = load_json_model(iCHO_path)

In [ ]:
amino_acids = {
    "arginine": ["EX_arg_L_e", "EX_arg_D_e"],
    "asparagine": ["EX_asn_L_e", "EX_asn_D_e"],
    "cysteine": ["EX_cys_L_e", "EX_cys_D_e", "EX_Lcystin_e"],
    "histidine": ["EX_his_L_e", "EX_his_D_e"],
    "isoleucine": ["EX_ile_L_e", "EX_ile_D_e"],
    "leucine": ["EX_leu_L_e", "EX_leu_D_e"],
    "lysine": ["EX_lys_L_e", "EX_lys_D_e"],
    "methionine": ["EX_met_L_e", "EX_met_D_e"],
    "phenylalanine": ["EX_phe_L_e", "EX_phe_D_e"],
    "proline": ["EX_pro_L_e", "EX_pro_D_e"],
    "threonine": ["EX_thr_L_e", "EX_thr_D_e"],
    "tryptophan": ["EX_trp_L_e", "EX_trp_D_e"],
    "valine": ["EX_val_L_e", "EX_val_D_e"]
}
for amino_acid in amino_acids:
    # ----- Setup initial bounds -----
    for exchange_reaction in iCHO.exchanges:
        exchange_reaction.bounds = -10, 10

    # Arginine
    iCHO.reactions.get_by_id('GAUGE-R00557').bounds = 0, 10
    iCHO.reactions.get_by_id('GAUGE-R10107').bounds = 0, 0
    iCHO.reactions.get_by_id('GAUGE-R00558').bounds = 0, 0
    iCHO.reactions.get_by_id('GLYAMDTRc').bounds = 0, 0
    iCHO.reactions.get_by_id('GAUGE-R10107').bounds = 0, 0
    iCHO.reactions.get_by_id('EX_valarggly_e').bounds = 0, 10
    iCHO.reactions.get_by_id('ARGSL').bounds = 0, 0

    # Asparigine
    iCHO.reactions.get_by_id('ASNS1').bounds = 0, 0

    # Cysteine
    iCHO.reactions.get_by_id('r0129').bounds = 0, 0
    iCHO.reactions.get_by_id('EX_cgly_e').bounds = 0, 0
    iCHO.reactions.get_by_id('AMPTASECG').bounds = -10, 0
    iCHO.reactions.get_by_id('AMPTASECGe').bounds = -10, 0
    iCHO.reactions.get_by_id('CYSTGL').bounds = -10, 0
    iCHO.reactions.get_by_id('EX_HC00250_e').bounds = 0, 0
    iCHO.reactions.get_by_id('EX_sfcys_e').bounds = 0, 0

    # Histidine
    iCHO.reactions.get_by_id('VALTRPVALr').bounds = 0,0 

    # Isoleucine - DONE
    iCHO.reactions.get_by_id('EX_CE2916_e').bounds = 0,0 
    iCHO.reactions.get_by_id('EX_CE2915_e').bounds = 0,0 
    iCHO.reactions.get_by_id('ILETA').bounds = 0, 10
    iCHO.reactions.get_by_id('ILETAm').bounds = 0, 10

    # Leucine
    iCHO.reactions.get_by_id('LEULEULAPc').bounds = 0,0 
    iCHO.reactions.get_by_id('EX_leugly_e').bounds = 0,0 
    iCHO.reactions.get_by_id('EX_glyleu_e').bounds = 0,0 
    iCHO.reactions.get_by_id('LEUTA').bounds = 0, 10 
    iCHO.reactions.get_by_id('LEUTAm').bounds = 0, 10 
    iCHO.reactions.get_by_id('EX_CE5797_e').bounds = 0, 0

    # Lysine
    iCHO.reactions.get_by_id('EX_biocyt_e').bounds = 0,0 

    # Methionine
    iCHO.reactions.get_by_id('METS').bounds = -10, 0 
    iCHO.reactions.get_by_id('BHMT').bounds = -10, 0 
    iCHO.reactions.get_by_id('GAUGE-R00648').bounds = 0, 10 
    iCHO.reactions.get_by_id('UNK2').bounds = -10, 0 
    iCHO.reactions.get_by_id('UNK3').bounds = -10, 0 
    iCHO.reactions.get_by_id('TYRA').bounds = -10, 0 
    #iCHO.reactions.get_by_id('GAUGE-R06895').bounds = 0, 0 # Curated by MR with 1 score / Erased from the reconstruction

    # Phenylalanine
    iCHO.reactions.get_by_id('EX_CE5786_e').bounds = 0, 0 
    iCHO.reactions.get_by_id('EX_pheleu_e').bounds = 0, 0 
    iCHO.reactions.get_by_id('EX_glyphe_e').bounds = 0, 0 
    iCHO.reactions.get_by_id('EX_CE2917_e').bounds = 0, 0 
    iCHO.reactions.get_by_id('EX_CE5786_e').bounds = 0, 0 
    iCHO.reactions.get_by_id('EX_CE5789_e').bounds = 0, 0 
    iCHO.reactions.get_by_id('EX_phpyr_e').bounds = 0, 0 

    # Proline
    iCHO.reactions.get_by_id('EX_glypro_e').bounds = 0, 10
    iCHO.reactions.get_by_id('EX_progly_e').bounds = 0, 10
    iCHO.reactions.get_by_id('P5CR').bounds = 0, 0
    iCHO.reactions.get_by_id('P5CRxm').bounds = 0, 0
    iCHO.reactions.get_by_id('P5CRx').bounds = 0, 0
    iCHO.reactions.get_by_id('P5CRm').bounds = 0, 0
    iCHO.reactions.get_by_id('r1453').bounds = 0, 10

    # Threonine
    #iCHO.reactions.get_by_id('THRS').bounds = 0,0  # Erased from the reconstruction


    # Tryptophan
    iCHO.reactions.get_by_id('NBAHH_ir').bounds = 0,0  

    # Valine
    iCHO.reactions.get_by_id('EX_valarggly_e').bounds = 0, 10
    iCHO.reactions.get_by_id('EX_vallystyr_e').bounds = 0, 10
    iCHO.reactions.get_by_id('VALTA').bounds = 0, 10
    iCHO.reactions.get_by_id('VALTAm').bounds = 0, 10
    iCHO.reactions.get_by_id('EX_valval_e').bounds = 0, 10
    # -----------------------------------------

    for exchange_reaction in iCHO.exchanges:
        if exchange_reaction.id in amino_acids[amino_acid]:
            exchange_reaction.bounds = 0, 10
            # print(amino_acid, exchange_reaction.id, amino_acids[amino_acid])
    sol = iCHO.optimize()
    print(amino_acid, sol.objective_value)


In [ ]:
amino_acids = {
    "alanine": ["EX_ala_B_e", "EX_ala_L_e", "EX_ala_D_e"],
    "arginine": ["EX_arg_L_e", "EX_arg_D_e"],
    "asparagine": ["EX_asn_L_e", "EX_asn_D_e"],
    "aspartic acid": ["EX_asp_L_e", "EX_asp_D_e"],
    "cysteine": ["EX_cys_L_e", "EX_cys_D_e"],
    "glutamine": ["EX_gln_L_e", "EX_gln_D_e"],
    "glutamic acid": ["EX_glu_L_e", "EX_glu_D_e"],
    "glycine": ["EX_gly_e"],
    "histidine": ["EX_his_L_e", "EX_his_D_e"],
    "isoleucine": ["EX_ile_L_e", "EX_ile_D_e"],
    "leucine": ["EX_leu_L_e", "EX_leu_D_e"],
    "lysine": ["EX_lys_L_e", "EX_lys_D_e"],
    "methionine": ["EX_met_L_e", "EX_met_D_e"],
    "phenylalanine": ["EX_phe_L_e", "EX_phe_D_e"],
    "proline": ["EX_pro_L_e", "EX_pro_D_e"],
    "serine": ["EX_ser_L_e", "EX_ser_D_e"],
    "threonine": ["EX_thr_L_e", "EX_thr_D_e"],
    "tryptophan": ["EX_trp_L_e", "EX_trp_D_e"],
    "tyrosine": ["EX_tyr_L_e", "EX_tyr_D_e"],
    "valine": ["EX_val_L_e", "EX_val_D_e"]
}

def test_auxotrophy(model, amino_acids):
    # Iterate through each amino acid and its corresponding exchange reactions
    for amino_acid, reactions in amino_acids.items():
        # Set the bounds for each exchange reaction
        for exchange in model.exchanges:
            if exchange.id in reactions:
                exchange.bounds = (0, 10)  # Allow secretion but not uptake
            else:
                exchange.bounds = (-10, 10)  # Allow uptake and secretion

        # Optimize the model for the current setup
        solution = model.optimize()
        if solution.status == "optimal":
            print(f"{amino_acid} biomass production: {solution.objective_value}")
        else:
            print(f"{amino_acid} optimization failed: {solution.status}")

In [ ]:
test_auxotrophy(iCHO, amino_acids)

In [ ]:
iCHO.metabolites.leu_L_c.summary()

#### Matrix Sparsity

#### Gene Deletion test

##### Essential for growth genes and comparison to (An optimised genome-wide, virus free CRISPR screen for mammalian cells)

##### Commparsion to other models

#### Biomass prediction using exp. data (TODO)

Incorporate uptake and secretion data and calculate the growth rate. Try ecFBA with enzyme trunover numbers. Reproduce Hooman's analsys with iCHO_v1